In [1]:
import pandas as pd
import numpy as np

In [2]:
## 雞肋公式沒卵用
## arg1: dataframe -- diary
## arg2: str -- desired column
## return: dataframe -- index=unique values in the columns; freq=frequency of each value
def var_freq(df,colName):
    varSer=df[colName].dropna().value_counts()
    varDF=pd.DataFrame(varSer)
    varDF.rename(columns={colName:'freq'},inplace=True)
    varDF.index.name=colName
    return varDF

In [22]:
## data= dataframe -- from hand typed csv
## export a csv of nodes and a csv of edges
## return
def to_graph(diary,path):
    ## clean the data
    diary.dropna(axis=[0,1],how='all',inplace=True)
    
    ## make people-event DataFrame
    ppl_event=diary[['Type','Participants']].dropna(subset=['Participants'])
    
    #### make event index
    event_iter=0
    event_idx=[]
    for boo in ppl_event['Type'].notnull():
        if boo:
            event_iter+=1
        event_idx.append(event_iter)
    
    ppl_event["Event"]=event_idx
    
    ## make a series of unique names as the index and columns of the adjacency matrix
    ppl_uniq=pd.Series(ppl_event['Participants'].value_counts().index).append(pd.Series("謝蘭生"),ignore_index=True)
    ppl_df=pd.DataFrame(ppl_uniq).set_index(0)
    #### export a csv of people as nodes
    ppl_df.to_csv(path+'/pplNodes.csv')
    
    ## initialize the adjacency matrix
    AdjMa=ppl_df.join(ppl_df.T,how='outer').fillna(0)
    
    ## Count frequency from event
    event=ppl_event.set_index('Event').drop('Type',axis=1)
    for i in np.arange(1,event_iter):
        if len(event.ix[i])>1:
            df=event.ix[i].set_index('Participants')
            AdjMa.ix[df.index,df.index]+=1
        i+=1
    #### diagonal -- 0
    np.fill_diagonal(AdjMa.values,0)
    #### for 謝蘭生
    freq=np.append(ppl_event['Participants'].value_counts().values,0)
    AdjMa["謝蘭生"]=freq
    AdjMa.ix["謝蘭生"]=freq
    ## export a csv of Adjacency Matrix
    AdjMa.to_csv(path+'/AdjMa.csv')
    
    
    return

In [6]:
diary=pd.read_csv("/Users/yumeng.zou/Google Drive/Freshyear/Summer/Research/1819 diary.csv")

In [23]:
to_graph(diary,"/Users/yumeng.zou/Downloads")